# Import Required Library

In [15]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from tqdm import tqdm  # Import tqdm for progress bar
from fake_useragent import UserAgent  # <-- Thêm thư viện giả lập User-Agent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import re
import ast  # để chuyển từ string thành list thật

#---------
from urllib.request import urlopen, Request

In [16]:
# Đọc nội dung file txt
with open('all_post_url.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Chuyển từ chuỗi sang list thực
url_list = ast.literal_eval(content)

# Tạo DataFrame từ list
df_url = pd.DataFrame(url_list, columns=['url'])

print(df_url.shape)  # In ra (số_url, 1)
df_url.head()  # Xem vài dòng đầu


(4235, 1)


,url
0,https://www.topcv.vn/viec-lam/korean-bridge-so...
1,https://www.topcv.vn/viec-lam/network-technica...
2,https://www.topcv.vn/viec-lam/gaming-sop-desig...
3,https://www.topcv.vn/viec-lam/lap-trinh-vien-b...
4,https://www.topcv.vn/viec-lam/chuyen-vien-kiem...


In [13]:
df_url.shape

(4184, 1)

In [14]:
list_url = df_url['url'].to_numpy()
for i in list_url[0:10]:
    print(i)

https://www.topcv.vn/viec-lam/technical-leader-truong-nhom-cong-nghe-luong-25-30-trieu-tai-ha-noi/1681869.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/solution-architect/1679125.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/senior-it-auditor-iso-27001/1584608.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/product-manager-product-owner-lending-product/1687717.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/2d-game-artist/1687263.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/pm-project-manager-brse-japanese-yeu-cau-tieng-nhat-n2-2-nam-kinh-nghiem-thu-nhap-tu-47-73-trieu/1693602.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/senior-java-developer-msb-1y560/1694269.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/microsoft-support-engineer-crm-erp-software-thu-nhap-hap-dan/1387005.html?ta_source=ITJobs_LinkDetail
https://www.topcv.vn/viec-lam/senior-developer-tieng-trung/1655938

In [ ]:
webdriver_path = "/usr/local/bin/chromedriver"
service = Service(webdriver_path)


{'identifier': '2'}

In [197]:
# Định nghĩa hàm clean_text ngoài vòng lặp để tối ưu
def clean_text(text):
    return '\n'.join(filter(None, (line.strip() for line in text.split('\n'))))

# Hàm xử lý retry cho brand
def get_soup_with_retry_brand(url, driver, retries=3, delay=3):
    for i in range(retries):
        try:
            driver.get(url)

            # Chờ tới khi DOM load xong
            WebDriverWait(driver, 10).until(
                lambda d: d.execute_script("return document.readyState") == "complete"
            )

            time.sleep(np.random.uniform(2, 6))  # Ngẫu nhiên để giống người
            return BeautifulSoup(driver.page_source, 'html.parser')

        except Exception as e:
            print(f"[Retry {i+1}/{retries}] Lỗi khi load {url}: {e}")
            time.sleep(delay)
    return None

# Hàm xử lý retry cho normal
def get_soup_with_retry_normal(url, driver, retries=3, delay=3):
    for i in range(retries):
        try:
            driver.get(url)

            # Chờ tới khi DOM load xong
            WebDriverWait(driver, 10).until(
                lambda d: d.execute_script("return document.readyState") == "complete"

            )# Chờ hiển thị class job-detail__info--title

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "job-detail__info--title"))
            )

            time.sleep(np.random.uniform(2, 6))  # Ngẫu nhiên để giống người
            return BeautifulSoup(driver.page_source, 'html.parser')

        except Exception as e:
            print(f"[Retry {i+1}/{retries}] Lỗi khi load {url}: {e}")
            time.sleep(delay)
    return None

# Scrape data not normal_url function

In [ ]:
# Hàm crawl dữ liệu cho từng URL
def crawl_data_for_each_normal_url(url, proxy_scraper_url=None):
            
    # Retry tải trang
    soup = get_soup_with_retry_normal(url, driver)
    if soup is None:
        # Nếu không tải được, ghi vào log và out hàmd 
        with open('error_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"{url}\n")
        return None

    # Job title
    job_title = 'Null'
    try:
        job_title = clean_text(soup.find('h1', class_='job-detail__info--title').get_text(separator=' '))
    except AttributeError:
        pass

    # Role
    role = 'Null'
    try:
        role = soup.find('div', class_='job-tags').find('a')['title']
        role = role.removeprefix("Chuyên môn ")
    except (AttributeError, TypeError):
        pass

    # Job description, location, time range, benefits
    location = time_range = benefits = 'Null'
    describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
    try:
        tag_div = soup.find('div', class_='job-description')
        items = tag_div.find_all('div', class_='job-description__item')
        for sub_items in items:
            name_tag = sub_items.find('h3').get_text()
            if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Quyền lợi': benefits = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Địa điểm làm việc': location = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())
            elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='job-description__item--content').get_text())

        # Xử lý trường hợp không có mô tả hoặc yêu cầu
        if describe != 'Null' and required != 'Null':
            description = describe + '\n' + required
        elif describe != 'Null':
            description = describe
        elif required != 'Null':
            description = required
        else:
            description = 'Null'

    except AttributeError:
        description = 'Null'

    # Salary, province, experience years
    salary = province = experience_years = 'Null'
    try:
        tag_spe = soup.find('div', class_='job-detail__info--sections')
        list_spe = tag_spe.select('div.job-detail__info--section-content-value')
        if len(list_spe) >= 3:
            salary, province, experience_years = [tag.get_text(strip=True) for tag in list_spe[:3]]
    except AttributeError:
        pass  # All values remain 'Null'

    # Company name
    company_name = 'Null'
    try:
        company_name = soup.find('a', class_='name').get_text(strip=True)
    except AttributeError:
        pass

    # Company URL
    url_company = 'Null'
    try:
        tag_url = soup.find('div', class_='job-detail__company--link')
        url_company = tag_url.find('a')['href']
    except (AttributeError, TypeError):
        pass

    # Other job details: level, academic level, job type
    level = academic_level = job_type = 'Null'
    try:
        tag_lat = soup.select('div.box-general-group-info-value')
        if len(tag_lat) >= 4:
            level = tag_lat[0].get_text(strip=True)
            academic_level = tag_lat[1].get_text(strip=True)
            job_type = tag_lat[3].get_text(strip=True)
    except AttributeError:
        pass  # All values remain 'Null'

    # Skills
    skills = 'Null'
    try:
        skills = soup.find('div', class_='box-category collapsed').find('div', class_='box-category-tags').get_text(separator=', ', strip=True)
    except AttributeError:
        pass

    # Deadline
    deadline = 'Null'
    try:
        deadline = soup.find('div', class_='job-detail__info--deadline').get_text(strip=True)
        deadline = deadline.removeprefix("Hạn nộp hồ sơ: ")
    except AttributeError:
        pass

    # Working language (hiện chưa crawl được)
    working_language = 'Null'

    # Thêm dòng vào DataFrame
    df.loc[len(df)] = [job_title, description, role, time_range, skills, location, province,
                        experience_years, salary, company_name, url_company, job_type,
                        working_language, deadline, academic_level, benefits, level, url]

    # Tự động lưu tạm vào CSV mỗi 100 dòng



# Scrape data brand_url function

In [199]:
# Hàm crawl dữ liệu cho từng BRAND_URL
def crawl_data_for_each_brand_url(url, proxy_scraper_url=None):

    # Retry tải trang
    soup = get_soup_with_retry_brand(url, driver)
    if soup is None:
        # Nếu không tải được, ghi vào log và tiếp tục URL tiếp theo
        with open('error_log.txt', 'a', encoding='utf-8') as f:
            f.write(f"{url}\n")
        return None
    # Job title
    job_title = 'Null'
    try:
        job_title = clean_text(soup.find('h2', class_='title').get_text(separator=' '))
    except AttributeError:
        pass

    # Role
    role = 'Null'
    try:
        role = soup.find('div', class_='job-tags').find('a')['title']
        role = role.removeprefix("Chuyên môn ")
    except (AttributeError, TypeError):
        pass

    # Job description, time range, benefits, province
    time_range = benefits = province = 'Null'
    describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
    try:
        tag_div = soup.find('div', class_='box-job-info')
        try:
            province = tag_div.find('div', class_='box-address').find('div').get_text(strip=True) # Lưu ý lúc này province không phải chỉ chứa mỗi tỉnh => Cần xử lý ở giai đoạn sau.
        except AttributeError:
            pass
        items = tag_div.find_all('div', class_='box-info')[1:]
        for sub_items in items:
            name_tag = sub_items.find('h2', class_='title').get_text()
            if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='content-tab').get_text())
            elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='content-tab').get_text())
            elif name_tag == 'Quyền lợi được hưởng': benefits = clean_text(sub_items.find('div', class_='content-tab').get_text())               
            elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='content-tab').get_text())

        # Xử lý trường hợp không có mô tả hoặc yêu cầu
        if describe != 'Null' and required != 'Null':
            description = describe + '\n' + required
        elif describe != 'Null':
            description = describe
        elif required != 'Null':
            description = required
        else:
            description = 'Null'

    except AttributeError:
        description = 'Null'

    # Salary, experience years, level, academic level, job type
    salary = experience_years = level = academic_level = job_type = 'Null'
    try:
        items = soup.find('div', class_='box-main').find_all('div', class_='box-item')
        for sub_items in items:
            name_tag = sub_items.find('strong').get_text()
            if name_tag == 'Mức lương': salary = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Hình thức làm việc ': job_type = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Cấp bậc ': level = clean_text(sub_items.find('span').get_text())           
            elif name_tag == 'Học vấn ': academic_level = clean_text(sub_items.find('span').get_text())
            elif name_tag == 'Kinh nghiệm ': experience_years = clean_text(sub_items.find('span').get_text())
    except AttributeError:
        pass  # All values remain 'Null'

    # Company name, location
    company_name = location = 'Null'
    try:
        company_name = soup.find('div', class_='footer-info-content footer-info-company-name').get_text(strip=True)
    except AttributeError:
        pass

    try:
        location = soup.find('div',class_='footer-info-content footer-info-company-name').find_next_sibling('div', class_='footer-info-content').get_text(strip=True)
    except AttributeError:
        pass


    # Company URL
    url_company = 'Null'
    try:
        url_company = soup.find('a', class_='navbar-diamond-company-brand')['href']
    except (AttributeError, TypeError):
        pass

    # Skills
    skills = 'Null'
    try:
        skills = soup.find('div', class_='box-skill box-category collapsed').find('div', class_='item').get_text(separator=', ', strip=True)
    except AttributeError:
        pass

    # Deadline
    deadline = 'Null' # ở các brand_url không cung cấp deadline

    # Working language (hiện chưa crawl được)
    working_language = 'Null'

    # Thêm dòng vào DataFrame
    df.loc[len(df)] = [job_title, description, role, time_range, skills, location, province,
                        experience_years, salary, company_name, url_company, job_type,
                        working_language, deadline, academic_level, benefits, level, url]




# Scrape_data_for_each_url

In [200]:
def scrape_data_for_each_url(url_list):
    for url in tqdm(url_list, desc="Processing URLs", unit="URL", ncols=100):
        try:
            if '/brand/' in url:
                crawl_data_for_each_brand_url(url)
            else: crawl_data_for_each_normal_url(url)
        except Exception as e:
            print(f"Lỗi khi xử lý URL: {url} - {e}")
            # Ghi lại lỗi vào file log
            with open('error_log.txt', 'a', encoding='utf-8') as f:
                f.write(f"{url}\n")
            time.sleep(5)  # Thêm thời gian chờ lâu hơn nếu gặp lỗi
        

        if len(df) % 100 == 0:
            df.to_csv("backup_topcv.csv", index=False)
    
    # Lưu toàn bộ DataFrame vào file CSV sau khi crawl xong
    df.to_csv("final_topcv.csv", index=False)

In [211]:
# Tạo dataframe mới để lưu dữ liệu đang cần thu thập
df = pd.DataFrame(columns=["Job Title", "Description", "Role", "Time Range", "Skills", "Location", "Province", 
                               "Experience Years", "Salary", "Company Name", "Company URL", "Job Type", 
                               "Working Language", "Deadline", "Academic Level", "Benefits", "Level", "URL"])

In [168]:
url_list = ['https://www.topcv.vn/viec-lam/senior-it-auditor-iso-27001/1584608.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/brand/trung-tam-anh-ngu-ila/tuyen-dung/erp-developer-j1692852.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/brand/nganhangthuongmaicophanquandoi/tuyen-dung/truong-phong-dam-bao-chat-luong-dich-vu-cntt-qa-manager-khoi-cong-nghe-thong-tin-ho25-145-j1694162.html?ta_source=ITJobs_LinkDetail']
scrape_data_for_each_url(url_list)

Processing URLs: 100%|███████████████████████████████████████████████| 3/3 [00:21<00:00,  7.28s/URL]


In [212]:
crawl_data_for_each_normal_url('https://www.topcv.vn/viec-lam/senior-it-auditor-iso-27001/1584608.html?ta_source=ITJobs_LinkDetail')

[Retry 1/3] Lỗi khi load https://www.topcv.vn/viec-lam/senior-it-auditor-iso-27001/1584608.html?ta_source=ITJobs_LinkDetail: Message: 
Stacktrace:
0   chromedriver                        0x000000010cb38168 chromedriver + 6177128
1   chromedriver                        0x000000010cb2f82a chromedriver + 6141994
2   chromedriver                        0x000000010c5b9e00 chromedriver + 417280
3   chromedriver                        0x000000010c60b4d7 chromedriver + 750807
4   chromedriver                        0x000000010c60b6f1 chromedriver + 751345
5   chromedriver                        0x000000010c65b6d4 chromedriver + 1078996
6   chromedriver                        0x000000010c63171d chromedriver + 907037
7   chromedriver                        0x000000010c658a22 chromedriver + 1067554
8   chromedriver                        0x000000010c6314c3 chromedriver + 906435
9   chromedriver                        0x000000010c5fd885 chromedriver + 694405
10  chromedriver                       

KeyboardInterrupt: 

In [209]:
df

,Job Title,Description,Role,Time Range,Skills,Location,Province,Experience Years,Salary,Company Name,Company URL,Job Type,Working Language,Deadline,Academic Level,Benefits,Level,URL
0,ERP Developer,Use MS SQL and related technologies to maintai...,Software Engineer,Thứ 2 - Thứ 6 (từ 09:00 đến 18:00),"Kiểm thử phần mềm, Kiến Trúc Phần Mềm, Phân Tí...","The Crest Residence, số 15, đường Trần Bạch Đằ...","- Hồ Chí Minh: Ila Thủ Thiêm, Thủ Đức",2 năm,Thoả thuận,Trung Tâm Anh Ngữ ILA,https://www.topcv.vn/brand/trung-tam-anh-ngu-i...,Toàn thời gian,Null,Null,Cao Đẳng trở lên,Insurances fully provided including Social Ins...,Nhân viên,https://www.topcv.vn/brand/trung-tam-anh-ngu-i...


In [202]:
crawl_data_for_each_brand_url('https://www.topcv.vn/brand/trung-tam-anh-ngu-ila/tuyen-dung/erp-developer-j1692852.html?ta_source=ITJobs_LinkDetail')

In [203]:
df

,Job Title,Description,Role,Time Range,Skills,Location,Province,Experience Years,Salary,Company Name,Company URL,Job Type,Working Language,Deadline,Academic Level,Benefits,Level,URL
0,ERP Developer,Use MS SQL and related technologies to maintai...,Software Engineer,Thứ 2 - Thứ 6 (từ 09:00 đến 18:00),"Kiểm thử phần mềm, Kiến Trúc Phần Mềm, Phân Tí...","The Crest Residence, số 15, đường Trần Bạch Đằ...","- Hồ Chí Minh: Ila Thủ Thiêm, Thủ Đức",2 năm,Thoả thuận,Trung Tâm Anh Ngữ ILA,https://www.topcv.vn/brand/trung-tam-anh-ngu-i...,Toàn thời gian,Null,Null,Cao Đẳng trở lên,Insurances fully provided including Social Ins...,Nhân viên,https://www.topcv.vn/brand/trung-tam-anh-ngu-i...


# Text

In [101]:
# test
url = 'https://www.topcv.vn/brand/nganhangthuongmaicophanquandoi/tuyen-dung/truong-phong-dam-bao-chat-luong-dich-vu-cntt-qa-manager-khoi-cong-nghe-thong-tin-ho25-145-j1694162.html?ta_source=ITJobs_LinkDetail'

# Tạo request với User-Agent giả lập trình duyệt
headers = {'User-Agent': 'Mozilla/5.0'}
req = Request(url, headers=headers)

# Gửi request
html = urlopen(req)
soup = BeautifulSoup(html, 'html.parser')

# job title
job_title = clean_text(soup.find('h2', class_='title').get_text(separator=' '))
print(job_title)
# role
role = soup.find('div', class_='job-tags').find('a')['title']
role = role.removeprefix("Chuyên môn ")
print(role)

# Job description, time range, benefits, province
time_range = benefits = province = 'Null'
describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
try:
    tag_div = soup.find('div', class_='box-job-info')
    try:
        province = tag_div.find('div', class_='box-address').find('div').get_text(strip=True) # Lưu ý lúc này province không phải chỉ chứa mỗi tỉnh => Cần xử lý ở giai đoạn sau.
    except AttributeError:
        pass
    items = tag_div.find_all('div', class_='box-info')[1:]
    for sub_items in items:
        name_tag = sub_items.find('h2', class_='title').get_text()
        if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='content-tab').get_text())
        elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='content-tab').get_text())
        elif name_tag == 'Quyền lợi được hưởng': benefits = clean_text(sub_items.find('div', class_='content-tab').get_text())               
        elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='content-tab').get_text())

    # Xử lý trường hợp không có mô tả hoặc yêu cầu
    if describe != 'Null' and required != 'Null':
        description = describe + '\n' + required
    elif describe != 'Null':
        description = describe
    elif required != 'Null':
        description = required
    else:
        description = 'Null'

except AttributeError:
    description = 'Null'

print(description)
print(time_range)
print(benefits)
print(province)

# Salary, experience years, level, academic level, job type
salary = experience_years = level = academic_level = job_type = 'Null'
try:
    items = soup.find('div', class_='box-main').find_all('div', class_='box-item')
    for sub_items in items:
        name_tag = sub_items.find('strong').get_text()
        if name_tag == 'Mức lương': salary = clean_text(sub_items.find('span').get_text())
        elif name_tag == 'Hình thức làm việc ': job_type = clean_text(sub_items.find('span').get_text())
        elif name_tag == 'Cấp bậc ': level = clean_text(sub_items.find('span').get_text())           
        elif name_tag == 'Học vấn ': academic_level = clean_text(sub_items.find('span').get_text())
        elif name_tag == 'Kinh nghiệm ': experience_years = clean_text(sub_items.find('span').get_text())
except AttributeError:
    pass  # All values remain 'Null'

print(salary)
print(job_type)
print(level)
print(academic_level)
print(experience_years)

# Company name, location
company_name = location = 'Null'
try:
    company_name = soup.find('div', class_='footer-info-content footer-info-company-name').get_text(strip=True)
except AttributeError:
    pass

try:
    location = soup.find('div',class_='footer-info-content footer-info-company-name').find_next_sibling('div', class_='footer-info-content').get_text(strip=True)
except AttributeError:
    pass

print(company_name)
print(location)

# Company URL
url_company = 'Null'
try:
    url_company = soup.find('a', class_='navbar-diamond-company-brand')['href']
except (AttributeError, TypeError):
    pass

print(url_company)

# Skills
skills = 'Null'
try:
    skills = soup.find('div', class_='box-skill box-category collapsed').find('div', class_='item').get_text(separator=', ', strip=True)
except AttributeError:
    pass

print(skills)


Trưởng Phòng Đảm Bảo Chất Lượng
Dịch Vụ CNTT - QA Manager - Khối Công Nghệ Thông Tin (HO25.145)
QA Engineer
Xây dựng tiêu chuẩn chất lượng dịch vụ Công nghệ thông tinXây dựng, cải tiến các quy trình, quy định về dịch vụ Công nghệ thông tinGiám sát và có phương pháp cải tiến liên tục để nâng cao chát lượng dịch vụ Công nghệ thông tinTuyển dụng, đào tạo, quản lý, phát triển nhân sự
Tốt nghiệp đại học chinh quy các trường Đại học chuyên ngành: CNTT, điện tử, toán tin, tài chính, ngân hàngHiểu biết quy trình, quy định phát triển phần mềmCó ít nhất 7 năm kinh nghiệm phát triển phần mềmHiểu biết sâu sắc về tiêu chuẩn chất lượng dịch vụ CNTT     + Có ít nhất 5 năm kinh nghiệm QA     + Có ít nhất 3 năm kinh nghiệm với vai trò Quản lý cấp tương đương.Ưu tiên: Có Kinh nghiệm BA/Test.Ưu tiên: Có Kinh nghiệm tại ngân hàng.Ưu tiên: Có chứng chỉ QA theo thông lệ Quốc tế (ISO, CMMI, ITIL)
Null
- Thu nhập : Thoã Thuận- Bảo hiểm- Du Lịch- Phụ cấp- Du lịch nước ngoài- Phụ cấp thâm niên- Công tác phí- Tă

In [192]:
# test
url = 'https://www.topcv.vn/brand/trung-tam-anh-ngu-ila/tuyen-dung/erp-developer-j1692852.html?ta_source=ITJobs_LinkDetail'

# Tạo request với User-Agent giả lập trình duyệt
headers = {'User-Agent': 'Mozilla/5.0'}
req = Request(url, headers=headers)

# Gửi request
html = urlopen(req)
soup = BeautifulSoup(html, 'html.parser')

# job title
job_title = clean_text(soup.find('h2', class_='title').get_text(separator=' '))
print(job_title)
# role
role = soup.find('div', class_='job-tags').find('a')['title']
role = role.removeprefix("Chuyên môn ")
print(role)

# Job description, time range, benefits, province
time_range = benefits = province = 'Null'
describe = required = 'Null'  # các biến để lưu mô tả và yêu cầu
try:
    tag_div = soup.find('div', class_='box-job-info')
    try:
        province = tag_div.find('div', class_='box-address').find('div').get_text(strip=True) # Lưu ý lúc này province không phải chỉ chứa mỗi tỉnh => Cần xử lý ở giai đoạn sau.
    except AttributeError:
        pass
    items = tag_div.find_all('div', class_='box-info')[1:]
    for sub_items in items:
        name_tag = sub_items.find('h2', class_='title').get_text()
        if name_tag == 'Mô tả công việc': describe = clean_text(sub_items.find('div', class_='content-tab').get_text())
        elif name_tag == 'Yêu cầu ứng viên': required = clean_text(sub_items.find('div', class_='content-tab').get_text())
        elif name_tag == 'Quyền lợi được hưởng': benefits = clean_text(sub_items.find('div', class_='content-tab').get_text())               
        elif name_tag == 'Thời gian làm việc': time_range = clean_text(sub_items.find('div', class_='content-tab').get_text())

    # Xử lý trường hợp không có mô tả hoặc yêu cầu
    if describe != 'Null' and required != 'Null':
        description = describe + '\n' + required
    elif describe != 'Null':
        description = describe
    elif required != 'Null':
        description = required
    else:
        description = 'Null'

except AttributeError:
    description = 'Null'

print(description)
print(time_range)
print(benefits)
print(province)

# Salary, experience years, level, academic level, job type
salary = experience_years = level = academic_level = job_type = 'Null'
try:
    items = soup.find('div', class_='box-main').find_all('div', class_='box-item')
    for sub_items in items:
        name_tag = sub_items.find('strong').get_text()
        if name_tag == 'Mức lương': salary = clean_text(sub_items.find('span').get_text())
        elif name_tag == 'Hình thức làm việc ': job_type = clean_text(sub_items.find('span').get_text())
        elif name_tag == 'Cấp bậc ': level = clean_text(sub_items.find('span').get_text())           
        elif name_tag == 'Học vấn ': academic_level = clean_text(sub_items.find('span').get_text())
        elif name_tag == 'Kinh nghiệm ': experience_years = clean_text(sub_items.find('span').get_text())
except AttributeError:
    pass  # All values remain 'Null'

print(salary)
print(job_type)
print(level)
print(academic_level)
print(experience_years)

# Company name, location
company_name = location = 'Null'
try:
    company_name = soup.find('div', class_='footer-info-content footer-info-company-name').get_text(strip=True)
except AttributeError:
    pass

try:
    location = soup.find('div',class_='footer-info-content footer-info-company-name').find_next_sibling('div', class_='footer-info-content').get_text(strip=True)
except AttributeError:
    pass

print(company_name)
print(location)

# Company URL
url_company = 'Null'
try:
    url_company = soup.find('a', class_='navbar-diamond-company-brand')['href']
except (AttributeError, TypeError):
    pass

print(url_company)

# Skills
skills = 'Null'
try:
    skills = soup.find('div', class_='box-skill box-category collapsed').find('div', class_='item').get_text(separator=', ', strip=True)
except AttributeError:
    pass

print(skills)


ERP  Developer
Software Engineer
Use MS SQL and related technologies to maintain and customize Microsoft Dynamics installations Collaborate with other Developers, Project Managers, Business Partners, End Users to identify, troubleshoot and implement solutionsConduct unit testing and debugging along with system and integration testingProduce technical specifications and software design documents Develop and support interfaces/integrations between D365 and related in-house or external business systemsProvide technical solution to troubleshoot issuesWilling to upskill/ use different framework if required
Must have University degree in Information Technology / Management Information Systems or equivalentMinimum 2 years of working experience in design and development of Dynamics AX components (Required: D365 FO. Preferred: AX 2012).Preferred experience:2+ years strong .Net / C#2+ years working on APIs and integration between multiple systemsPHP and/or Python is an added advantage.
Thứ 2 - T

In [ ]:
temp = ['https://www.topcv.vn/brand/nganhangthuongmaicophanquandoi/tuyen-dung/truong-phong-dam-bao-chat-luong-dich-vu-cntt-qa-manager-khoi-cong-nghe-thong-tin-ho25-145-j1694162.html?ta_source=ITJobs_LinkDetail'] 
crawl_data_for_each_url(temp)

In [76]:
df

,Job Title,Description,Role,Time Range,Skills,Location,Province,Experience Years,Salary,Company Name,Company URL,Job Type,Working Language,Deadline,Academic Level,Benefits,Level,URL
